In [7]:
# from utility.MoleculeParser import MoleculeData
# from utility.QMUQUBO import QMUQUBO
# from utility.AnnealerOptimizer import Annealer
# from utility.ResultProcess import ResultParser
# import time

# timestamp = time.strftime("%Y%m%d-%H")

# import packages, including those to connect with AWS BraKet

import numpy as np
import pandas as pd
import math
import os
import glob

from braket.aws import AwsDevice
from braket.ocean_plugin import BraketSampler, BraketDWaveSampler

In [8]:
# function to read in .ct file and give a list of known structure stems:

def actual_stems(seq_ss, seq_ps):
    
    with open(subdirectory+"/"+seq_ss) as file:
        lines = file.readlines()
    
    with open(subdirectory+"/"+seq_ps) as file:
        fasta_lines = file.readlines()
    
    rna = fasta_lines[1]
    
    stems_actual = []

    sip = False                       # stem in progress?
    sl = 0                            # stem length
    last_line = [0, 0, 0, 0, 0, 0]    # initiate last line

    for i in range(0, len(lines)):
        line = lines[i].strip().split()
        if (int(line[4]) != 0 and sip == False):
            sip = True
            temp = [int(line[0]), int(line[4])]
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl += 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl += 2
        if (int(line[4]) != 0 and sip == True and (int(last_line[4])-int(line[4]) == 1)):
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl += 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl += 2
        if (int(line[4]) == 0 and sip == True):
            sip = False
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            sl = 0
        if ((int(last_line[4])-int(line[4]) != 1) and int(last_line[4]) != 0  and sip == True):
            temp.append(sl)
            if temp[1] > temp[0]:
                stems_actual.append(temp)
            temp = [int(line[0]), int(line[4])]
            sl = 0
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('C' or 'c')) or (rna[i] == ('C' or 'c') and rna[int(line[4])-1] == ('G' or 'g')):
                sl = 3
            if (rna[i] == ('G' or 'g') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('G' or 'g')) or (rna[i] == ('A' or 'a') and rna[int(line[4])-1] == ('U' or 'u')) or (rna[i] == ('U' or 'u') and rna[int(line[4])-1] == ('A' or 'a')):
                sl = 2
        
        last_line = line
        
    return stems_actual

In [9]:
# function to evaluate the energy of the known structure under the model Hamiltonian:

def energy(stems_actual, pkp):
    
    cl = 1
    cb = 1
    k = 0
    
    pseudoknots_actual = potential_pseudoknots(stems_actual, pkp)
    cost = 0
    mu = max(list(map(list, zip(*stems_actual)))[2])
    
    for i in range(0, len(stems_actual)):
        cost += cl*((stems_actual[i][2]**2)-2*mu*stems_actual[i][2]+mu**2)-cb*(stems_actual[i][2]**2)
        for j in range(i+1, len(stems_actual)):
            cost -= 2*cb*stems_actual[i][2]*stems_actual[j][2]*pseudoknots_actual[k][2]
            k += 1
    
    return cost

In [10]:
# function to generate list of potential stem pairs that form pseudoknots:

def potential_pseudoknots(stems_potential, pkp):

    pseudoknots_potential = []
    pseudoknot_penalty = pkp

    for i in range(len(stems_potential)):
        for j in range(i + 1, len(stems_potential)):
            
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            i_a = stem1[0]
            j_a = stem1[1]
            i_b = stem2[0]
            j_b = stem2[1]
    
            pseudoknot = [i,j,1]
    
            if (i_a < i_b and i_b < j_a and j_a < j_b) or (i_b < i_a and i_a < j_b and j_b < j_a):
        
                pseudoknot[2] = pseudoknot_penalty
    
            pseudoknots_potential.append(pseudoknot)
            
    return pseudoknots_potential

In [11]:
# function to generate list of stem pairs that overlap:

def potential_overlaps(stems_potential):
    
    overlaps_potential = []
    overlap_penalty = 1e6

    for i in range(len(stems_potential)):
        for j in range(i+1, len(stems_potential)):
    
            stem1 = stems_potential[i]
            stem2 = stems_potential[j]
    
            overlap = [i, j, 0]
    
            stem1_cspan1 = set(range(stem1[1]-int(stem1[2])+1, stem1[1]+1))
            stem2_cspan1 = set(range(stem2[1]-int(stem2[2])+1, stem2[1]+1))
            
            stem1_cspan2 = set(range(stem1[0], stem1[0]+int(stem1[2])))
            stem2_cspan2 = set(range(stem2[0], stem2[0]+int(stem2[2])))
    
            if (len(stem1_cspan1 & stem2_cspan1) != 0) or (len(stem1_cspan2 & stem2_cspan2) != 0)  or (len(stem1_cspan1 & stem2_cspan2) != 0) or (len(stem1_cspan2 & stem2_cspan1) != 0):
        
                overlap[2] = overlap_penalty
        
            overlaps_potential.append(overlap)
            
    return overlaps_potential

In [12]:
# function to read in .fasta file and generate list of potential stems at least 3 base-pairs long:

def potential_stems(seq_ps):
    
    with open(subdirectory+"/"+seq_ps) as file:
        lines = file.readlines()
    
    rna = lines[1]
    
    matrix = np.zeros((len(rna),len(rna)))
    for diag in range(0, len(matrix)):
        for row in range(0, len(matrix)-diag):
            col = row + diag
            base1 = rna[row]
            base2 = rna[col]
            if row != col:
                if ((base1 == ("A" or "a")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("A" or "a"))) or ((base1 == ("G" or "g")) and (base2 == ("U" or "u"))) or ((base1 == ("U" or "u")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 2
                if ((base1 == ("G" or "g")) and (base2 == ("C" or "c"))) or ((base1 == ("C" or "c")) and (base2 == ("G" or "g"))):
                    matrix[row][col] = 3
    
    stems_potential = []
    mu = 0

    for row in range(0, len(matrix)):
        for col in range (row, len(matrix)):
            if row != col:
                if matrix[row][col] != 0:
                    temp_row = row
                    temp_col = col
                    stem = [row+1,col+1,0]
                    length_N = 0
                    length_H = 0
                    while (matrix[temp_row][temp_col] != 0) and (temp_row != temp_col):
                        length_N+=1
                        length_H+=matrix[temp_row][temp_col]
                        temp_row+=1
                        temp_col-=1
                        if length_N >= 3:
                            stem[2] = int(length_H)
                            stems_potential.append(stem.copy())
                    if length_H > mu:
                        mu = length_H
    
    return [stems_potential, mu, rna, len(rna)]

In [ ]:
# function to generate the Hamiltonian of a given RNA structure from potential stems, overlaps, and pseudoknots:

def model(stems_potential, pseudoknots_potential, overlaps_potential, mu):
    
    L = {}
    Q = {}
    cl = 1
    cb = 1
    k = 0

    for i in range(0, len(stems_potential)):
        L[str(i)] = cl*((stems_potential[i][2]**2)-2*mu*stems_potential[i][2]+mu**2)-cb*(stems_potential[i][2]**2)
        for j in range(i+1, len(stems_potential)):
            Q[(str(i), str(j))] = -2*cb*stems_potential[i][2]*stems_potential[j][2]*pseudoknots_potential[k][2]+overlaps_potential[k][2]
            k += 1
    
    return L, Q

In [13]:

# pk = ["wPKs", "woutPKs"]
pk = ["wPKs"]
# s  = ["s", "m", "l"]
s  = ["s"]
p  = [-1.0, -0.5, 0.0, 0.5, 1.0]
pl = ["n1", "np5", "0", "pp5", "p1"]

data = []

for a in range(0, len(pk)):
    for b in range(0, len(s)):
        for c in range(0, len(pl)):
            
            subdirectory = './known_structures/'+pk[a]+'/'+s[b]

            ct = [f for f in os.listdir(subdirectory) if f.endswith('.ct.txt')]
            fasta = [f for f in os.listdir(subdirectory) if f.endswith('.fasta.txt')]

            bprna_id = []
            size = []
            pks = []
            pk_penalty = []

            penalty = pl[c]

            for i in range(0, len(ct)):
                bprna_id.append(ct[i].split('.')[0])
                size.append(subdirectory.split("/")[3])
                if subdirectory.split("/")[2] == "wPKs":
                    pks.append("T")
                else:
                    pks.append("F")
                pk_penalty.append(penalty)
            
            stems_a    = []
            energies_a = []
            stems_p    = []
            pks_p      = []
            ols_p      = []
            models     = []

            for index in range(0, len(ct)):
                stems_a.append(actual_stems(ct[index], fasta[index]))
                energies_a.append(energy(stems_a[index], p[c]))
                stems_p.append(potential_stems(fasta[index]))
                pks_p.append(potential_pseudoknots(stems_p[index][0], p[c]))
                ols_p.append(potential_overlaps(stems_p[index][0]))
                models.append(model(stems_p[index][0], pks_p[index], ols_p[index], stems_p[index][1]))
                                
            print(models)
            problem = []
            # print(pk_penalty)

NameError: name 'model' is not defined

In [ ]:
# loop over all structures, submitting to D-Wave, recording predicted stems and energies, recording comparisons between known and precicted structures:

pk = ["wPKs", "woutPKs"]
s  = ["s", "m", "l"]
p  = [-1.0, -0.5, 0.0, 0.5, 1.0]
pl = ["n1", "np5", "0", "pp5", "p1"]

data = []

for a in range(0, len(pk)):
    for b in range(0, len(s)):
        for c in range(0, len(pl)):
            
            subdirectory = './known_structures/'+pk[a]+'/'+s[b]

            ct = [f for f in os.listdir(subdirectory) if f.endswith('.ct.txt')]
            fasta = [f for f in os.listdir(subdirectory) if f.endswith('.fasta.txt')]

            bprna_id = []
            size = []
            pks = []
            pk_penalty = []

            penalty = pl[c]

            for i in range(0, len(ct)):
                bprna_id.append(ct[i].split('.')[0])
                size.append(subdirectory.split("/")[3])
                if subdirectory.split("/")[2] == "wPKs":
                    pks.append("T")
                else:
                    pks.append("F")
                pk_penalty.append(penalty)
            
            stems_a    = []
            energies_a = []
            stems_p    = []
            pks_p      = []
            ols_p      = []
            models     = []

            for index in range(0, len(ct)):
                stems_a.append(actual_stems(ct[index], fasta[index]))
                energies_a.append(energy(stems_a[index], p[c]))
                stems_p.append(potential_stems(fasta[index]))
                pks_p.append(potential_pseudoknots(stems_p[index][0], p[c]))
                ols_p.append(potential_overlaps(stems_p[index][0]))
                models.append(model(stems_p[index][0], pks_p[index], ols_p[index], stems_p[index][1]))
                                
            problem = []

            for i in range(0, len(ct)):
                problem.append(dimod.BinaryQuadraticModel(models[i][0], models[i][1], vartype = 'BINARY', offset = 0.0))    
            
            stems_f = []
            min_time = []

            for i in range(0, len(ct)):
                try:
                    sampleset = sampler.sample(problem[i], num_reads=1)
                    min_time.append("placeholder")
                
                    for datum in sampleset.data(['sample', 'energy', 'num_occurrences']):
                        results = datum.sample
                        predicted_energy = datum.energy
    
                    f_stems = []

                    for j in range(0, len(results)):
                        if results[str(j)] == 1:
                            f_stems.append(stems_p[i][0][j])
            
                    stems_f.append([f_stems, predicted_energy])
                except:
                    print("no embedding found, skipping...")
            
            metrics_1 = []
            metrics_2 = []

            for index in range(0, len(ct)):
                try:
                    metrics_1.append(evaluation_1(stems_a[index], stems_f[index][0]))
                    metrics_2.append(evaluation_2(stems_a[index], stems_f[index][0]))
                except:
                    print("no structure found, skipping...")
            
            headers = ["bprna_id", "sequence", "length", "size", "pks", "pk_penalty", "stems_actual", "energy_actual", "stems_predicted", "energy_predicted", "time_to_solution", "sensitivity_bp", "specificity_bp", "sensitivity_b", "specificity_b"]
            data.append(pd.DataFrame(list(zip(bprna_id, list(map(list, zip(*stems_p)))[2], list(map(list, zip(*stems_p)))[3], size, pks, pk_penalty, stems_a, energies_a, list(map(list, zip(*stems_f)))[0], list(map(list, zip(*stems_f)))[1], min_time, list(map(list, zip(*metrics_1)))[0], list(map(list, zip(*metrics_1)))[1], list(map(list, zip(*metrics_2)))[0], list(map(list, zip(*metrics_2)))[1])), columns = headers))

# Step 1: Prepare Data

In this part, we load the raw molecule data for experiment.
The [117 ligand](http://www.rcsb.org/ligand/117) was 
put in the repository. We assign the relative 
path to **raw_path**.
The **s3_bucket** and **prefix** are used to store the 
optimization results. We can use the one created with the 
cloudformation for convenience.

In [ ]:
# initial parameters for experiment data
s3_bucket = f"xxxx" # change to the name of bucket created in your deployment
prefix = "xxxx" # the name of the folder in the bucket

raw_path = './molecule-data/117_ideal.mol2' # the mol2 file for this experiment

In [ ]:
mol_data = MoleculeData(raw_path, 'qmu')

data_path = mol_data.save("latest")

num_rotation_bond = mol_data.bond_graph.rb_num
print(f"You have loaded the raw molecule data and saved as {data_path}. \n\
This molecule has {num_rotation_bond} rotable bond")

After running this block, the processed data 
will be saved as **qmu_117_ideal_data_latest.pickle**
and **data_path** will be updated. We can see that this 
molecule has 23 rotatable bonds.

# Step 2: Build Model

In this part, we build the Quadratic Unconstrained 
Binary Optimization (QUBO) model for molecular unfolding.

First, we set the following parameters and 
initialize the QMUQUBO object. 

<center>

| Parameter | Description | Value |
|--- |--- |--- |
|A | penalty scalar |300|
|hubo_qubo_val | energy penalty of make_quadratic() |200|
|M | number of torsions for molecular unfolding| [1, max number of rotatable bonds] |
|D| angle precision of rotation| 8|
|method| the method of building model| 'pre-calc': calculate the score in advance|

 </center>

We use the 'pre-calc' method 
to build the model. This molecule has 23 rotatable bonds and 
we only test 2 of them, so we set the **M** to 2. And we want 
the angle to become $45^o$, so we set the **D** to 8 
(i.e., $8=360^o/45^o$). The **A** and **hubo_qubo_val** are 
test from experiments. 

In [ ]:
# initial the QMUQUBO object
init_param = {}
method = ['pre-calc']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

In [ ]:
# set the parameters for model
model_param = {}
# parameters
num_rotation_bond = mol_data.bond_graph.rb_num

method = 'pre-calc'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [1,2,3]
model_param[method]['D'] = [8]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]

qmu_qubo.build_model(**model_param)

We can use the following method to check the properties of 
model. This way, we can build many models conveniently. 
After that, we save the model and update the value of 
**model_path**.

In [ ]:
# describe the model parameters
model_info = qmu_qubo.describe_model()

In [ ]:
# save the model
model_path = qmu_qubo.save("latest")

print(f"You have built the QUBO model and saved it as {model_path}")

# Step 3: Optimize Configuration

In this part, we use SA and QA to find the optimized configuration of molecular unfolding.
At first, we load the model file using **QMUQUBO** object

In [ ]:
qmu_qubo_optimize = QMUQUBO.load(model_path)

In [ ]:
model_info = qmu_qubo_optimize.describe_model()

We can see the parameters of this model, with M equaling 1,2 or 3, D equaling 8, 
A equaling 300 and hubo_qubo_val equaling 200. 
Actually, we can contain multiple models in this file just 
by giving multiple values for one parameter when creating models.

Actually, we can contain multiple models in this file just 
by giving multiple values for one parameter when creating models.
Then, we need use **model_name** to get the model for experiments.

In [ ]:
# get the model you want to optimize
M = 2
D = 8
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

We can see that we want to carry out experiment with the QUBO model with M equaling 2.
 After that, we set the parameters for optimization.

| Parameter | Description | Value |
|--- |--- |--- |
|method | annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
|shots| number of reads, refer to [dwave-sa](https://docs.ocean.dwavesys.com/projects/neal/en/latest/reference/generated/neal.sampler.SimulatedAnnealingSampler.sample.html#neal.sampler.SimulatedAnnealingSampler.sample) and [dwave-qa](https://amazon-braket-ocean-plugin-python.readthedocs.io/en/latest/_apidoc/braket.ocean_plugin.braket_sampler.html) for details |1 to 10,000|
|bucket | the s3 bucket to store your results | - |
|prefix | the name of the folder in your s3 bucket | - |
|device | the arn name to run your quantum annealing| 'arn:aws:braket:::device/qpu/d-wave/Advantage_system4' <br> 'arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6'|

Then, we can run the SA for this problem:

In [ ]:
method = 'neal-sa'

optimizer_param = {}
optimizer_param['shots'] = 1000

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

In [ ]:
sa_optimize_result = sa_optimizer.fit()

We can tell that we set the number of shots for SA to 1000. 
The result is saved as the local file **./sa_result.pickle.**
Alternatively, we can use QA to solve this problem:

In [ ]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1000
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

In this QA, we set the number of shots to 1000 and 
choose the 
[Advantage_System4.1](https://docs.dwavesys.com/docs/latest/doc_physical_properties.html)
as the QPU. In addition, the results are saved to your bucket automatically and you 
can get the task id for future process. 

In [ ]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

In [ ]:
qa_task_id = qa_optimizer.get_task_id()
print(f"task id is {qa_task_id}")

Finally, we can compare the execution time between SA and QA :

In [ ]:
print(f"dwave-sa run time {sa_optimize_result['time']}")
print(f"dwave-qa run time {qa_optimize_result['time']}")

We can tell from the image that SA needs more than 150 seconds 
and QA needs more than 5 seconds to find 
solution.

We sometimes get the best result that occurs only once.

![OneTimeQA](../../../docs/en/images/one-time-qa.png)

This does not always indicate an error. It is actually the characteristic of the problem or how the problem 
is formulated. Because we have different linear and quadratic terms that vary by many orders of magnitude. If we 
set change value of A to some smaller number, like 10 or 100, more occurrences of the best answer will be observed. 
However, these answers usually break the constraints. For more information about this phenomenon, please refer to this 
[Link](https://support.dwavesys.com/hc/en-us/community/posts/1500000698522-Number-of-occurrences-?input_string=number%20occurance).

# Step 4: PostProcess Result

In this part, we post process the optimizing results for evaluation and visualization.
At first, we prepare the following parameters:

| Parameter | Description | Value |
|--- |--- |--- |
|method | annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
|raw_path| the path for the original molecule file| './molecule-data/117_ideal.mol2' in this example |
|data_path| the path for the processed molecule file| './qmu_117_ideal_data_latest.mol2' in this example |
|bucket | the s3 bucket to store your results | - |
|prefix | the name of the folder in your s3 bucket | - |
|task_id | the id for your quantum annealing task| '2b5a3b05-1a0e-443a-852c-4ec422a10e59' in this example |

Then we can run the post-process using **ResultParser** object for SA:

In [ ]:
method = "neal-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

In [ ]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_result_filepath, sa_result_json = sa_process_result.save_mol_file(f"{timestamp}")

print(f"result path is {sa_result_filepath}, and result optimization file path is {sa_result_json}")

In [ ]:
sa_process_result.parameters

In the first block, we can see the **local time**
for SA is more than 150 seconds. 
With the **generate_optimize_pts()** method, the final 3D 
points after unfolding will be generated and saved as json file and mol2 files. The last 
block shows the optimizing results which are also stored in json files. 
It shows that the optimized result gains 
1.0006x increase in volume. The value for **unfolding_results** indicates 
that the rotatable bond 15 should rotate $270^o$ ($360/8*(7-1)$) and 
the rotatable bond 14 should rotate $0^o$ ($360/8*(1-1)$).
At the same time, you can run the post-process for QA:

In [ ]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

we can see that there many types of time metrics for running QA.
This task has the **local time** of 5 s, which means the time between calling the api and 
getting the annealing result. The **task time** time is the metric from the json file in 
bucket. We can also see the **qpu total time** and **qpu access time** representing the 
actual time running in the QPU. Please refer to [Operation and Timing](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)
for details.

In [ ]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_result_filepath, qa_result_json = qa_process_result.save_mol_file(f"{timestamp}")
print(f"result path is {qa_result_filepath}, and result optimization file path is {qa_result_json}")

In [ ]:
qa_process_result.parameters

In same way, the optimized results are translated the 3D points and saved 
as local json and mol2 files. The result indicates that QA gains 
1.0006x increase in 
volume.

Finally, We can open folders for the optimized results:

![optimize-results](../../../docs/en/images/optimize-results.png)

 <center>Optimize Results</center>

We can see the json result and mol2 file of SA and QA are 
stored in this place. If we carry out more 
experiments, more results with time stamp are 
stored incrementally. 
For visualization, we can use the following method to see the original the molecule and unfolding molecule.

In [ ]:
# this shows the original molecule
qa_process_result.InteractView(raw_path, size=800)

In [ ]:
# copy the variable for the path of unfolding molecule, we can see the unfolding results. this also works for sa_process_result.
qa_process_result.InteractView(qa_result_filepath, size=800)

Alternatively, we can upload the mol2 file into 
[online viewer tool](https://www.rcsb.org/3d-view) 
to see the result:

![visual](../../../docs/en/images/visualization.png)

 <center>Visualization</center>